In [2]:
import json # Parsing JSON
import tqdm # Processbar
import pandas as pd # Dataframes
from pathlib import Path # Working with files

Get a list of all candidates

In [2]:
with open("data/kandidat_urls", "r") as f: # open file with urls
    kandidat_urls = f.read() # save file in variable
    kandidat_list = kandidat_urls.split("\n") # make the file into a list, without new lines

Extract demographic information

In [3]:
demo_list = [] # make empty list
for kandidat in kandidat_list: # go through all candidates
    kandidat_file = f"data/kandidater/{kandidat}.json" # file name for candidate

    with open(kandidat_file, "r") as f: # open candidate file
        kandidat_json = json.load(f) # load the json
            
    kandidat_data = { # json index for relevant information:
        "ID" : kandidat_json["props"]["pageProps"]["candidate"]["UrlKey"], 
        "first_name" : kandidat_json["props"]["pageProps"]["candidate"]["Firstname"],
        "last_name" : kandidat_json["props"]["pageProps"]["candidate"]["LastName"],
        "gender" : kandidat_json["props"]["pageProps"]["candidate"]["Gender"],
        "birth" : kandidat_json["props"]["pageProps"]["candidate"]["Birthdate"],
        "party" : kandidat_json["props"]["pageProps"]["candidate"]["CurrentPartyCode"],
        "profession" : kandidat_json["props"]["pageProps"]["candidate"]["Profession"],
        "education" : kandidat_json["props"]["pageProps"]["candidate"]["Education"]}
    
    #måske også sæt svar ind her?
    
    demo_list.append(kandidat_data) # append data for one candidate to the rest
    
demo = pd.DataFrame(demo_list) # make dict into dataframe

demo.to_csv(Path('data/kandidat_demo.csv')) # save dataframe

In [4]:
col_names = []
for i in range(1,26):
    col_names += [f"answer_{i}", f"info_{i}"]

answers = pd.DataFrame(
    columns = col_names,
    index = kandidat_list)

for kandidat in kandidat_list: # go through all candidates
    kandidat_answers = [] # empty list for one candidate
    kandidat_file = f"data/kandidater/{kandidat}.json" # file name for candidate

    with open(kandidat_file, "r") as f:
        kandidat_json = json.load(f) # get the json
    
    for i in range(25): # go through all 25 questions
        
        if len(kandidat_json["props"]["pageProps"]["candidateAnswers"]) == 0: # if the candidate did not answer the question
            answer_i = None 
            info_i = None
        else:
            answer_i = kandidat_json["props"]["pageProps"]["candidateAnswers"][i]["Answer"] # save the answer
            info_i = kandidat_json["props"]["pageProps"]["candidateAnswers"][i]["Info"] # save potential comments to answer
        
        kandidat_answers += [answer_i, info_i]

    answers.loc[kandidat,:] = kandidat_answers

answers.to_csv(Path('data/kandidat_answers.csv')) # save dataframe